In [42]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import numpy as np
import pandas as pd
import geopandas as gpd
import tgess
import os
from sklearn.preprocessing import StandardScaler
from tqdm.auto import tqdm
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, \
                            r2_score, mean_absolute_percentage_error
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import WhiteKernel, RBF, RationalQuadratic
from modAL.models import ActiveLearner
from tgess.src.data_science.plots import *
from tgess.src.data_science.experiment import *
import seaborn as sns
import matplotlib.pyplot as plt
from enum import Flag, IntFlag, auto
from datetime import datetime, timedelta

sns.set_theme()
sns.set_style("whitegrid")
tqdm.pandas()

In [43]:
data_path = "../data/raw/AGB/TropiSAR"
csv_path = "CSV_TropiSAR_AfriSAR"
shp_path = "VECTOR_TropiSAR_AfriSAR"

In [174]:
afrisar_shp_df = gpd.read_file(os.path.join(data_path, shp_path, "Polygons_Calibration_points_AfriSAR_North.shp"))

In [175]:
afrisar_shp_df.crs

<Projected CRS: EPSG:32632>
Name: WGS 84 / UTM zone 32N
Axis Info [cartesian]:
- E[east]: Easting (metre)
- N[north]: Northing (metre)
Area of Use:
- name: Between 6°E and 12°E, northern hemisphere between equator and 84°N, onshore and offshore. Algeria. Austria. Cameroon. Denmark. Equatorial Guinea. France. Gabon. Germany. Italy. Libya. Liechtenstein. Monaco. Netherlands. Niger. Nigeria. Norway. Sao Tome and Principe. Svalbard. Sweden. Switzerland. Tunisia. Vatican City State.
- bounds: (6.0, 0.0, 12.0, 84.0)
Coordinate Operation:
- name: UTM zone 32N
- method: Transverse Mercator
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [76]:
afrisar_df = pd.read_csv(os.path.join(data_path, csv_path, "AfriSAR_plotbased_AGB.csv"), sep=";")

In [77]:
afrisar_df.describe()

,sw_x,sw_y,nw_x,nw_y,se_x,se_y,ne_x,ne_y,agb_fph,agb_chv,agb_loc,BA,WD,Ndens
count,339.000000,3.390000e+02,339.000000,3.390000e+02,339.000000,3.390000e+02,339.000000,3.390000e+02,339.000000,339.000000,339.000000,339.000000,339.000000,339.000000
mean,629222.544467,7.176456e+06,629198.274905,7.176504e+06,629270.252845,7.176480e+06,629245.983283,7.176528e+06,300.082596,274.010619,265.644543,23.010356,0.602786,371.435841
std,86751.110623,4.381073e+06,86747.119817,4.381066e+06,86749.763782,4.381087e+06,86745.773726,4.381080e+06,174.963048,167.295495,160.232432,11.819265,0.128322,150.798738
min,535118.753900,5.949727e+04,535118.230300,5.954725e+04,535168.751200,5.949613e+04,535168.227600,5.954612e+04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,546908.151300,6.465449e+04,546908.413100,6.472949e+04,546983.150850,6.465514e+04,546983.325400,6.472992e+04,194.450000,172.500000,166.450000,17.763090,0.540556,331.250000
50%,597934.366200,9.787250e+06,597887.360700,9.787289e+06,597973.163200,9.787297e+06,597926.606900,9.787336e+06,314.400000,284.800000,278.000000,24.260667,0.632021,412.500000
75%,674484.269950,9.917804e+06,674482.315400,9.917847e+06,674524.254350,9.917811e+06,674523.137450,9.917850e+06,400.850000,361.300000,349.750000,29.261574,0.685511,474.000000
max,791161.417000,9.980732e+06,791162.376900,9.980782e+06,791211.407800,9.980731e+06,791212.367700,9.980781e+06,948.300000,990.200000,918.200000,56.612916,0.855589,688.000000


In [78]:
afrisar_df["x"] = (afrisar_df["sw_x"]+afrisar_df["nw_x"]+afrisar_df["se_x"]+afrisar_df["ne_x"])/4
afrisar_df["y"] = (afrisar_df["sw_y"]+afrisar_df["nw_y"]+afrisar_df["se_y"]+afrisar_df["ne_y"])/4

In [79]:
afrisar_df.head()

,Site,Area_code,Plot_code,Scale,sw_x,sw_y,nw_x,nw_y,se_x,se_y,...,cred_loc_97.5,TCH,H50,GF20,STDV,BA,WD,Ndens,x,y
0,RABI,RAB01h,RABI,1ha,597926.1577,9786898.412,597848.1146,9786960.936,597988.6820,9786976.455,...,233.2,19.66271739,18.41500473,0.600020014,7.858543006,20.796339,0.661042,454.0,597918.398300,9.786969e+06
1,RABI,RAB02h,RABI,1ha,597848.1146,9786960.936,597770.0716,9787023.460,597910.6389,9787038.979,...,316.2,24.02936268,23.35000229,0.356164384,8.872869831,23.858093,0.677997,500.0,597840.355250,9.787031e+06
2,RABI,RAB03h,RABI,1ha,597770.0716,9787023.460,597692.0286,9787085.985,597832.5959,9787101.504,...,333.4,25.55422702,26.93000412,0.255374463,8.277211893,27.675211,0.644401,541.0,597762.312225,9.787094e+06
3,RABI,RAB04h,RABI,1ha,597692.0286,9787085.985,597613.9855,9787148.509,597754.5528,9787164.028,...,324.9,26.81398204,27.71125221,0.157305617,7.207124513,26.336228,0.645411,430.0,597684.269175,9.787156e+06
4,RABI,RAB05h,RABI,1ha,597613.9855,9787148.509,597535.9425,9787211.033,597676.5098,9787226.552,...,368.5,26.58188375,26.1099987,0.212812813,8.064963946,27.681182,0.623711,453.0,597606.226125,9.787219e+06


In [80]:
afrisar_df["Site"].unique()

array(['RABI', 'MONDAH', 'MABOUNIE', 'LOPE'], dtype=object)

In [81]:
afrisar_df.columns

Index(['Site', 'Area_code', 'Plot_code', 'Scale', 'sw_x', 'sw_y', 'nw_x',
       'nw_y', 'se_x', 'se_y', 'ne_x', 'ne_y', 'agb_fph', 'cred_fph_2.5',
       'cred_fph_97.5', 'agb_chv', 'cred_chv_2.5', 'cred_chv_97.5', 'agb_loc',
       'cred_loc_2.5', 'cred_loc_97.5', 'TCH', 'H50', 'GF20', 'STDV', 'BA',
       'WD', 'Ndens', 'x', 'y'],
      dtype='object')

In [82]:
!pip install utm

In [83]:
import utm

In [119]:
from pyproj import Transformer, Proj, transform

In [179]:
def get_lat_lon(x):
    if x["Site"]=="MONDAH":
        # AfriSAR North
        zone_number=32
        zone_letter="N"
        zone="32N"
    elif x["Site"] in ['RABI', 'MABOUNIE', 'LOPE']:
        # AfriSAR South
        zone_number=32
        zone_letter="S"
        zone="32S"
    else:
        # TropiSAR
        zone_number=22
        zone_letter="N"
        zone="22N"
#     transformer = Transformer.from_crs("UTM:{}{}".format(zone_number, zone_letter), "EPSG:4326", authority="EPSG")
    p1 = Proj(proj='utm',zone=zone)
    p2 = Proj(init='epsg:4326')

    x["zone"] = zone
    x["lon"], x["lat"] = transform(p1, p2, x["x"], x["y"])
    
    #Proj(proj='utm',zone=zone)(x["x"], x["y"], inverse=True)

#     x["lat"], x["lon"] = utm.to_latlon(x["x"], x["y"], zone_number=zone_number, zone_letter=zone_letter)
    return x

In [180]:
afrisar_df = afrisar_df.progress_apply(get_lat_lon, axis=1)

  0%|          | 0/339 [00:00<?, ?it/s]

/home/vneuteboom/anaconda3/envs/master-thesis/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/home/vneuteboom/anaconda3/envs/master-thesis/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/home/vneuteboom/anaconda3/envs/master-thesis/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/home/vneuteboom/anaconda3/envs/master-thesis/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/home/vneuteboom/anaconda3/e

/home/vneuteboom/anaconda3/envs/master-thesis/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/home/vneuteboom/anaconda3/envs/master-thesis/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/home/vneuteboom/anaconda3/envs/master-thesis/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/home/vneuteboom/anaconda3/envs/master-thesis/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/home/vneuteboom/anaconda3/e

/home/vneuteboom/anaconda3/envs/master-thesis/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/home/vneuteboom/anaconda3/envs/master-thesis/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/home/vneuteboom/anaconda3/envs/master-thesis/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/home/vneuteboom/anaconda3/envs/master-thesis/lib/python3.7/site-packages/ipykernel_launcher.py:22: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
/home/vneuteboom/anaconda3/e

In [185]:
temp_df = afrisar_df[afrisar_df["zone"]=="32N"]

In [186]:
import folium
from folium import plugins
f_map = folium.Map(location=[np.median(afrisar_df["lat"]), np.median(afrisar_df["lon"])], zoom_start=4.4)\
                   
# convert to (n, 2) nd-array format for heatmap
# points = df[['lat', 'lon']]

# # plot heatmap
# f_map.add_child(plugins.HeatMap(points, radius=7))

for point in temp_df[["lat", "lon"]].values:
    folium.CircleMarker(list(point), fill_color="darkblue", radius=0.5, weight=0).add_to(f_map)
f_map.fit_bounds([[-90, -180], [90, 180]])
display(f_map)

In [165]:
afrisar_df

,Site,Area_code,Plot_code,Scale,sw_x,sw_y,nw_x,nw_y,se_x,se_y,...,GF20,STDV,BA,WD,Ndens,x,y,lat,lon,zone
0,RABI,RAB01h,RABI,1ha,597926.1577,9786898.412,597848.1146,9786960.936,597988.6820,9786976.455,...,0.600020014,7.858543006,20.796339,0.661042,454.0,597918.398300,9.786969e+06,87.916673,33.899728,32S
1,RABI,RAB02h,RABI,1ha,597848.1146,9786960.936,597770.0716,9787023.460,597910.6389,9787038.979,...,0.356164384,8.872869831,23.858093,0.677997,500.0,597840.355250,9.787031e+06,87.917475,33.888764,32S
2,RABI,RAB03h,RABI,1ha,597770.0716,9787023.460,597692.0286,9787085.985,597832.5959,9787101.504,...,0.255374463,8.277211893,27.675211,0.644401,541.0,597762.312225,9.787094e+06,87.918277,33.877792,32S
3,RABI,RAB04h,RABI,1ha,597692.0286,9787085.985,597613.9855,9787148.509,597754.5528,9787164.028,...,0.157305617,7.207124513,26.336228,0.645411,430.0,597684.269175,9.787156e+06,87.919079,33.866812,32S
4,RABI,RAB05h,RABI,1ha,597613.9855,9787148.509,597535.9425,9787211.033,597676.5098,9787226.552,...,0.212812813,8.064963946,27.681182,0.623711,453.0,597606.226125,9.787219e+06,87.919881,33.855823,32S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
334,LOPE,LPG01q4,LPG01,0.25ha,786484.8848,9980730.873,786485.8447,9980780.864,786534.8756,9980729.914,...,0.058376649,7.665762426,47.177938,0.672541,328.0,786510.360150,9.980755e+06,87.430053,95.563747,32S
335,LOPE,LPG02q1,LPG02,0.25ha,791110.4664,9976131.430,791111.4262,9976181.421,791160.4571,9976130.470,...,0.028411365,6.281579624,30.155579,0.604779,148.0,791135.941675,9.976156e+06,87.385985,94.717455,32S
336,LOPE,LPG02q2,LPG02,0.25ha,791111.4262,9976181.421,791112.3861,9976231.412,791161.4170,9976180.461,...,0.078,7.540955811,47.425213,0.753370,196.0,791136.901550,9.976206e+06,87.386010,94.727249,32S
337,LOPE,LPG02q3,LPG02,0.25ha,791160.4571,9976130.470,791161.4170,9976180.461,791210.4479,9976129.510,...,0.034386246,6.859129115,21.777674,0.610202,224.0,791185.932450,9.976155e+06,87.385538,94.718000,32S


In [162]:
afrisar_df["lat"].describe()

count    339.000000
mean      63.928422
std       39.017214
min        0.538503
25%        0.585281
50%       87.917475
75%       87.920277
max       88.324608
Name: lat, dtype: float64